# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv in df
available_weather_data = pd.read_csv("../output_data/cities.csv")
available_weather_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,butaritari,75,KI,1586040213,78,3.07,172.79,28.45,5.60
1,1,barentu,5,ER,1586040214,44,15.11,37.59,22.58,1.12
2,2,sao filipe,53,IN,1586040214,29,20.51,76.21,25.99,1.48
3,3,cape town,0,CL,1586040214,35,-36.41,-71.90,22.00,4.60
4,4,marawi,14,PH,1586040214,56,8.00,124.29,24.03,0.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#Coordinates
location_data = available_weather_data[["Lat","Lng"]].astype(float)
#print(location_data)

#Humidity info
humidity_data = available_weather_data["Humidity"].astype(float)
#print(humidity_data)

#Figure Layout
figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

heatmap = gmaps.heatmap_layer(location_data, weights=humidity_data,
                              dissipating=False, max_intensity=100,
                              point_radius=4)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#* Narrow down the DataFrame to find your ideal weather condition. For example:
#  * A max temperature lower than 80 degrees but higher than 70. (weather data in Celcius so converting to Fahrehit 80 = 26.67 // 70 = 21.11)
#  * Wind speed less than 10 mph. (data in metrics so converting to Km - 16.09)
#  * Zero cloudiness.
#  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_weather = available_weather_data.loc[(available_weather_data["Max Temp"] < 26.67) 
                                           & (available_weather_data["Max Temp"] > 21.11) 
                                           & (available_weather_data["Wind Speed"] < 16.09)
                                           & (available_weather_data["Cloudiness"] == 0)]
ideal_weather = ideal_weather.dropna(how='any')
ideal_weather.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,cape town,0,CL,1586040214,35,-36.41,-71.90,22.00,4.60
44,44,punta arenas,0,PH,1586040224,67,8.99,125.34,26.38,1.77
76,76,arraial do cabo,0,BR,1586040231,41,-22.84,-46.65,25.00,2.60
105,105,namibe,0,AO,1586040238,69,-15.20,12.15,23.64,3.11
204,204,bubaque,0,GW,1586040266,70,11.28,-15.83,26.27,4.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_name = []

#reset index for ideal_weather
ideal_weather.reset_index(inplace=True)
del ideal_weather['index']

#loop through locations to find hotels
for i in range(len(ideal_weather)):
    #get lat & Lng for each location
    lat = ideal_weather.loc[i]['Lat']
    lng = ideal_weather.loc[i]['Lng']
    
    #request parameters
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    #hotel data for all hotels in 5000 meters
    hotel_data = requests.get(base_url, params=params)
    jsn = hotel_data.json()
    #print(jsn)
    
    try: 
        hotel_name.append(jsn['results'][0]['name'])
    
    except:
        hotel_name.append("")


In [7]:
ideal_weather["Hotel Name"] = hotel_name
ideal_weather = ideal_weather.dropna(how='any')
ideal_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,3,cape town,0,CL,1586040214,35,-36.41,-71.90,22.00,4.60,San Carlos
1,44,punta arenas,0,PH,1586040224,67,8.99,125.34,26.38,1.77,Nasipit
2,76,arraial do cabo,0,BR,1586040231,41,-22.84,-46.65,25.00,2.60,Bragança Paulista
3,105,namibe,0,AO,1586040238,69,-15.20,12.15,23.64,3.11,Namibe
4,204,bubaque,0,GW,1586040266,70,11.28,-15.83,26.27,4.42,Bubaque
5,220,pasni,0,PK,1586040278,52,25.26,63.47,25.44,3.07,Pasni
6,241,atar,0,MR,1586040283,23,20.52,-13.05,22.45,1.70,Atar
7,244,jardim,0,BR,1586040284,53,-21.48,-56.14,22.76,1.37,Jardim
8,271,campana,0,AR,1586040290,64,-34.17,-58.96,23.89,2.10,Campana
9,279,lindi,0,TZ,1586040292,87,-10.00,39.72,24.48,2.19,Lindi


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…